# Lecture 5
Serialization of data and use of external application API (Application Programmable Interface)

Often Python scripts are used to process data in consequence creating new data for further use. The processing part can itself be computationally involving and being able to store data in the easily accessible form is very convenient. Especially so if the processing part takes days (but even hours) and access to data is required at some latter stages of the project. One, obvious solution is to store resulting data in simple CSV type files, but this approach is limited, especially if data structure becomes complex.
For example, we might be storing time history of velocity profiles $(u,v,w)$ through a flow field retrieved from a number of locations and performed across snapshots in time ... . In general we have a lot of complex data to deal with, and would like to store it in some organized manner, to be retrieved and worked on latter. For this we can use **serialization**, which in the simplest terms means to store data. The concept comes from the need to communicate different application forming a system and working possibly across different platforms (or simply form game saves ...).

Here we will illustrate how to **store** and **retrieve** data we process and produce. Consider a process where we have a number of snapshots from computations (data could be data from experiments) describing some arbitrary vector field in time (or positions of a robot, some mappings of its surroundings, readings from sensors etc.). We will store such data within objects of the following class:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
class Case:
    def __init__(self, fname):
        self.file_name = fname # File name from which we will read
        
        self.time = 0 # solution time
        self.number_of_pointst = 0 # number of data points

        self.profile_pos = np.array([0,0,0]) # position of the profile cut

        self.Y = np.array # Y coordinate
        self.u = np.array # Components of velocity acros Y
        self.v = np.array
        self.w = np.array

So we have a simple class that will hold $(u,v,w)$ components of a vector obtained at $time$ and at a point with $Y$ the coordinate across the cut. Lets now populate it with some made up data:

In [ ]:
clist = []
c = Case("file_0.dat") # some file we read from
c.time = 0
c.number_of_pointst = 1000
c.profile_pos[0] = 5 # move at x=5
c.profile_pos[1] = 10 # move at y=5
c.Y = np.linspace(-1,1,100)
# Normally u,v,w are read from file, here made up
c.u = 1-c.Y**2
c.v = c.Y * 0
c.w = c.Y * 0
clist.append(c) # add to a list

In [ ]:
plt.plot(c.u, c.Y)

Let have a second one:

In [ ]:
c = Case("filw_1.dat") # some file we read from
c.time = 1
c.number_of_pointst = 1000
c.profile_pos[0] = 5 # move at x=5
c.profile_pos[1] = 10 # move at y=5
c.Y = np.linspace(-1,1,100)
# Normally u,v,w are read from file, here made up
c.u = 1-c.Y**3
c.v = c.Y * 0
c.w = c.Y * 0
clist.append(c) # add to a list

In [ ]:
for c in clist:
    plt.plot(c.u, c.Y)
plt.xlim(0,2)
plt.ylim(-1,1)
plt.grid()

usually there will be more data, the structure might be more complex, or some additional process will be used to do something with the data etc.

For data storage we will use the `msgpack`, which is a serialization format for use in data exchange across different languages, but not only. Here is the project:  
https://github.com/msgpack/msgpack-python  
and its API documentation:  
https://msgpack-python.readthedocs.io/en/latest/api.html  

In short, hands on How to Use we will require an instruction on how to handle data. How to encode and pack it and how to unpack and decode it. So we need to add encoding as well as decoding functions. Those function will accept an object of our data storage class and inform `msgpack` what to do with it. The whole process is in fact quite simple. We just do:

In [ ]:
import msgpack

To import an appropriate module, and:

In [ ]:
type(clist[0]) == int

In [ ]:
"""Encoding function"""
def enc_full(obj):
    if type(obj) == Case: # make sure obj is of some type
        code = 1 # A flag determining the type
        # pack and return
        return msgpack.ExtType(code, msgpack.packb([obj.file_name, obj.time, obj.number_of_pointst,
                                             obj.profile_pos.tostring(),
                                             obj.Y.tostring(),
                                             obj.u.tostring(), obj.v.tostring(), obj.w.tostring()]))
    raise TypeError # Rise an error if type not handled

And we have the encoding part. Now, all simple data types go as they are (file_name, time ...) and Numpy arrays need to be converted to strings. The `packb` simply packs an object into an appropriate format. One comment here is that we are not packing the `Case` object itself, but rather a list (note `[ ]`) of attributes such object has. The `code` identifies the stored type, since we could have more than one class to deal with the `code` would let us distinguish recovered data.
We can now store this data to a file, simply with:

In [ ]:
msgpack.dump(clist, open( 'aaa.dat', "wb" ), default=enc_full)

The `"wb"` lets us store this data in binary format. We will need to recover the data appropriatly with `"rb"`. See if the file was created, and look inside.

Now restart the Python interpreter and try to recover the data we put on disk. The `Case` class still needs to be defined, so in case you restarted the interpreter, do run the appropriate cell.  
To decode we will need a decoding function. It will contain instructions on how to rebuild objects from storage, create an object, feed it with data and return:

In [ ]:
import msgpack

In [ ]:
def dnc(code, data):
    if code == 1: # a flag determining the type
        # call the unpack on the data passed in here
        file_name, time, num_pts, pos, Y, u, v, w = msgpack.unpackb(data)
        
        # make the object
        c = Case(file_name)
        
        # feed it with data
        c.time = time
        c.number_of_pointst = num_pts
        # Note that numpy arrays need to be handled in a speciall way
        c.profile_pos = np.frombuffer(pos)
        c.Y = np.frombuffer(Y)
        c.u = np.frombuffer(u)
        c.v = np.frombuffer(v)
        c.w = np.frombuffer(w)
        
        # return the object outside
        return c
    # in case the type is either a build in or other
    return msgpack.ExtType(code, data)

We have just one class, and its code is simply `1`. `dnc()` will be passed this `code` as well as the binary data, which will be unpacked and returned as a tuple.
It is up to us to create a new object and populate it with data and than to return it.

Lifting data from storage is now simple. Note that we need to pass the decoding function to be used by `load()`, the file name and `"rb"` since we read binary data:

In [ ]:
recovered_list = msgpack.load(open( 'aaa.dat', "rb" ), ext_hook=dnc)

Now a copy of our data should be in `recovered_list`:

In [ ]:
for c in recovered_list:
    plt.plot(c.u, c.Y)
plt.xlim(0,2)
plt.ylim(-1,1)
plt.grid()

## Working with API
Various application provide some form of interface to interact with it. Python API becomes more and more a standard. How to connect to, how to use and to what it might be beneficial is very much problem dependant. For this I decided to show you connection to the Open Source visualization package ParaView. I use it to visualize CFD data, but it also used in other fields of engineering.

So the first step is to get your hands on the API, it will come as a Python module, it might need to be compiled, or be available as a software package. Here, I have the module compiled to a library placed in `/opt/paraview/lib/python3.8/`. Also I know I will need some of the Numpy functionality to process data:

In [ ]:
import sys
sys.path.append('/opt/paraview/lib/python3.8/site-packages/')
sys.path.append('/opt/paraview/lib/python3.8/')
sys.path.append('/opt/paraview/lib/')

from paraview.simple import *
from paraview.numeric import fromvtkarray

import numpy as np

from vtk.util.numpy_support import vtk_to_numpy
from vtk.numpy_interface import dataset_adapter as dsa
from vtk.numpy_interface import algorithms as algs

import matplotlib.pyplot as plt

What can be done with the API is very much up to the providing application and the level of documentation. In the case of Open Source project documentation is often lacking. This is the case with ParaView (was some time ago, could be better now). In what follows I will load a flow field snapshot and plot velocity profiles at selected positions:

1. Read the data

In [ ]:
geom = VisItTecplotBinaryReader(FileName=['geom.plt'])
geom.PointArrayStatus = ['u', 'v', 'w']

In [ ]:
plot_over_line = PlotOverLine(Input=geom, Source='High Resolution Line Source')
plot_over_line.Source.Point1 = [0.0, -1.0, 12.0]
plot_over_line.Source.Point2 = [0.0, 1.0, 12.0]

In [ ]:
pl = servermanager.Fetch(plot_over_line)

pos = fromvtkarray(pl.GetPoints().GetData())
valsU = fromvtkarray(pl.GetPointData().GetScalars("u"))
valsU = np.hstack([val for val in valsU])
valsV = fromvtkarray(pl.GetPointData().GetScalars("v"))
valsV = np.hstack([val for val in valsV])
valsW = fromvtkarray(pl.GetPointData().GetScalars("w"))
valsW = np.hstack([val for val in valsW])

And here we have velocity components and positions, just what we need to make a nice plot. One of the things that one may, or not learn from documentation is that `pos` contains a transposed coordinates:

In [ ]:
plt.plot(pos.T[1], valsU)
plt.plot(pos.T[1], valsV)
plt.grid()

In [ ]:
plt.plot(pos.T[1], valsW)
plt.grid()

We can modify the direction data is taken from:

In [ ]:
plot_over_line.Source.Point1 = [0.0, 0.0, 0.0]
plot_over_line.Source.Point2 = [0.0, 0.0, 25.13]

In [ ]:
pl = servermanager.Fetch(plot_over_line)

pos = fromvtkarray(pl.GetPoints().GetData())
valsU = fromvtkarray(pl.GetPointData().GetScalars("u"))
valsU = np.hstack([val for val in valsU])
valsV = fromvtkarray(pl.GetPointData().GetScalars("v"))
valsV = np.hstack([val for val in valsV])
valsW = fromvtkarray(pl.GetPointData().GetScalars("w"))
valsW = np.hstack([val for val in valsW])

Now plotting along $z$ coordinate:

In [ ]:
plt.plot(pos.T[2], valsW)
plt.grid()